### Supplementary Figure 2 | High-variance immune genes show coordinated longitudinal expression patterns
Longitudinal trajectories of highly variable immune-related genes reveal coordinated expression dynamics across vaccination.

In [1]:
############################################################
## 0) Setup
############################################################
suppressPackageStartupMessages({
  library(DESeq2)
  library(dplyr)
  library(tidyr)
  library(readr)
  library(stringr)
  library(ggplot2)
  library(ggrepel)
  library(fgsea)
  library(msigdbr)
  library(tidyverse)
  library(data.table)
  library(AnnotationDbi)
  library(ComplexHeatmap)
  library(circlize)
  library(SummarizedExperiment)
  library(BiocParallel)
  library(org.Hs.eg.db)
  library(matrixStats)
  library(pheatmap)
  library(grid)
})

if (.Platform$OS.type == "windows") {
  BiocParallel::register(
    BiocParallel::SnowParam(workers = max(1, parallel::detectCores() - 1))
  )
} else {
  BiocParallel::register(
    BiocParallel::MulticoreParam(workers = max(1, parallel::detectCores() - 1))
  )
}

set.seed(42)
dir.create("figs", showWarnings = FALSE)
out_dir <- "results_summary"
dir.create(out_dir, showWarnings = FALSE, recursive = TRUE)

se_all_full <- readRDS("results_paired_all_timepoints/se_paired_all_timepoints.rds")
print(se_all_full)

############################################################
## 1) Config and helpers
############################################################
visits      <- c("V1D0","V1D7","V2D0","V2D7")
top_n_each  <- 100
padj_thr    <- 0.05
exclude_pat <- "^(RPL|RPS|MRPL|MRPS)"   # exclude ribosomal-like for plotting

`%||%` <- function(a, b) if (is.null(a)) b else a

is_ENSG_like <- function(x) {
  grepl("^ENS[A-Z]*[0-9]", x %||% "")
}

ens2sym <- function(ids) {
  ids2 <- sub("\\.\\d+$", "", ids)
  m <- AnnotationDbi::mapIds(
    org.Hs.eg.db,
    keys      = unique(ids2),
    keytype   = "ENSEMBL",
    column    = "SYMBOL",
    multiVals = "first"
  )
  sym <- unname(m[ids2])
  sym
}

.get_tp <- function(cd) {
  tp <- NULL
  if ("m_time_point" %in% names(cd)) tp <- as.character(cd$m_time_point)
  if (is.null(tp) && "m_timepoint" %in% names(cd)) tp <- as.character(cd$m_timepoint)
  if (is.null(tp)) {
    src <- if ("sample_id" %in% names(cd)) cd$sample_id else rownames(cd)
    tp <- stringr::str_extract(src, "V[12]D\\d+")
  }
  tp
}

.clean_group <- function(x) {
  x <- toupper(as.character(x))
  x[x %in% c("SX","SARC","SARCOID","SARCOIDOSIS","SX?")] <- "SX"
  x[x %in% c("HC","HEALTHY","HEALTHY CONTROL","CONTROL","HC?")] <- "HC"
  factor(x, levels = c("HC","SX"))
}

## lncRNA genes to mark with "*"
lnc_genes <- c("MALAT1","PSMB8-AS1","SNHG32", "TALAM1", "ZNF292")  # extend as needed

## DESeq2 for a single visit
de_visit <- function(se, visit, padj_thr = 0.05) {
  cd <- as.data.frame(colData(se))
  tp <- .get_tp(cd)
  keep <- !is.na(tp) & tp == visit
  stopifnot(sum(keep) >= 4)

  se_v <- se[, keep, drop = FALSE]
  cdv  <- as.data.frame(colData(se_v))
  cdv$g <- .clean_group(cdv$m_group)
  if (any(table(cdv$g) < 2)) stop("Need ≥ 2 per group at ", visit)

  cts <- assay(se_v, "counts")
  cts[!is.finite(cts)] <- 0
  cts <- as.matrix(round(cts))
  storage.mode(cts) <- "integer"
  stopifnot(identical(colnames(cts), rownames(cdv)))

  dds <- DESeqDataSetFromMatrix(
    countData = cts,
    colData   = cdv,
    design    = ~ g
  )
  dds <- DESeq(dds, parallel = FALSE)

  rn <- resultsNames(dds)
  coef_name <- if ("g_SX_vs_HC" %in% rn) {
    "g_SX_vs_HC"
  } else {
    rn[grepl("^g_.*_vs_.*$", rn)][1]
  }

  res <- results(dds, name = coef_name, alpha = padj_thr)

  df <- as.data.frame(res)
  df$gene_id <- rownames(df)
  df$symbol  <- ens2sym(df$gene_id)

  df <- df %>%
    dplyr::select(gene_id, symbol, log2FC = log2FoldChange, padj) %>%
    dplyr::arrange(padj)

  list(dds = dds, de = df)
}

############################################################
## 2) DE per visit, union of top DEGs (non-ribosomal, non-ENSG)
############################################################
stopifnot(exists("se_all_full"), is(se_all_full, "SummarizedExperiment"))
stopifnot("counts" %in% assayNames(se_all_full))

union_ids <- character(0)
per_visit <- setNames(vector("list", length(visits)), visits)

for (v in visits) {
  out <- de_visit(se_all_full, v, padj_thr)
  per_visit[[v]] <- out

  de <- out$de %>%
    dplyr::filter(
      is.finite(padj),
      padj < padj_thr
    ) %>%
    dplyr::arrange(padj, dplyr::desc(abs(log2FC)))

  de <- de[
    !grepl(exclude_pat, de$symbol %||% "") &
      !is_ENSG_like(de$symbol),
    ,
    drop = FALSE
  ]

  union_ids <- unique(c(union_ids, head(de$gene_id, top_n_each)))
}
stopifnot(length(union_ids) > 0)

############################################################
## 3) VST and ΔVST (SX − HC) per visit
############################################################
cd_all   <- as.data.frame(colData(se_all_full))
tp_all   <- .get_tp(cd_all)
keep_all <- tp_all %in% visits
se_sub   <- se_all_full[, keep_all, drop = FALSE]

cts_all <- as.matrix(round(assay(se_sub, "counts")))
storage.mode(cts_all) <- "integer"

dds_all <- DESeqDataSetFromMatrix(
  countData = cts_all,
  colData   = colData(se_sub),
  design    = ~ 1
)

vsd <- vst(dds_all, blind = TRUE)

vst_mat <- assay(vsd)
vst_mat <- vst_mat[union_ids, , drop = FALSE]

syms <- ens2sym(rownames(vst_mat))
keep_sym <- !is.na(syms) & syms != "" & !is_ENSG_like(syms)
vst_mat  <- vst_mat[keep_sym, , drop = FALSE]
syms     <- syms[keep_sym]
rownames(vst_mat) <- syms

cd_sub          <- as.data.frame(colData(se_sub))
cd_sub$Visit    <- .get_tp(cd_sub)
cd_sub$Group    <- .clean_group(cd_sub$m_group)

visit_means <- lapply(visits, function(v) {
  idx_v  <- cd_sub$Visit == v
  sx_idx <- idx_v & cd_sub$Group == "SX"
  hc_idx <- idx_v & cd_sub$Group == "HC"
  stopifnot(sum(sx_idx) > 0, sum(hc_idx) > 0)
  rowMeans(vst_mat[, sx_idx, drop = FALSE]) -
    rowMeans(vst_mat[, hc_idx, drop = FALSE])
})
names(visit_means) <- visits

mat_delta <- do.call(cbind, visit_means)
mat_delta <- mat_delta[, visits, drop = FALSE]

############################################################
## 4) Union heatmap (optional, with clearer legend)
############################################################
mat_z <- t(scale(t(mat_delta)))
rownames(mat_z) <- rownames(mat_delta)
colnames(mat_z) <- visits

col_fun <- colorRamp2(c(-2, 0, 2), c("#2166AC", "#F7F7F7", "#B2182B"))

## Column annotation (Dose and Day)
dose <- ifelse(grepl("^V1", colnames(mat_z)), "Dose 1", "Dose 2")
day  <- ifelse(grepl("D0$", colnames(mat_z)), "Day 0", "Day 7")

ha_cols <- list(
  Dose = c("Dose 1" = "#444444", "Dose 2" = "#AAAAAA"),
  Day  = c("Day 0"  = "#ffffff", "Day 7"  = "#cccccc")
)

top_anno_union <- HeatmapAnnotation(
  Dose = dose,
  Day  = day,
  col  = ha_cols,
  annotation_name_side = "left",
  simple_anno_size = unit(4, "mm")
)


############################################################
## 5) Top-variance 50 heatmap, split by Up/Down at V1D0
############################################################
K <- 100
mat_ord <- mat_delta[, c("V1D0","V1D7","V2D0","V2D7"), drop = FALSE]

vars <- apply(mat_ord, 1, var, na.rm = TRUE)
top_idx <- order(vars, decreasing = TRUE)[seq_len(min(K, length(vars)))]
mat_top <- mat_ord[top_idx, , drop = FALSE]

mat_top_z <- t(scale(t(mat_top)))
rownames(mat_top_z) <- rownames(mat_top)
colnames(mat_top_z) <- c("V1D0","V1D7","V2D0","V2D7")

## Mark lncRNAs with "*"
genes <- rownames(mat_top_z)
genes_marked <- ifelse(genes %in% lnc_genes, paste0(genes, "*"), genes)
rownames(mat_top_z) <- genes_marked

row_group <- ifelse(mat_top_z[, "V1D0"] >= 0, "Up at V1D0", "Down at V1D0")

## Column annotation for this matrix
dose_top <- ifelse(grepl("^V1", colnames(mat_top_z)), "Dose 1", "Dose 2")
day_top  <- ifelse(grepl("D0$", colnames(mat_top_z)), "Day 0", "Day 7")

top_anno_top50 <- HeatmapAnnotation(
  Dose = dose_top,
  Day  = day_top,
  col  = ha_cols,
  annotation_name_side = "left",
  simple_anno_size = unit(4, "mm")
)

col_fun2 <- colorRamp2(c(-2, 0, 2), c("#2166AC", "#F7F7F7", "#B2182B"))

ht_top50 <- Heatmap(
  mat_top_z,
  name = "legend",
  col  = col_fun2,
  cluster_rows    = TRUE,  # dendrograms retained within each split
  row_split       = factor(row_group,
                           levels = c("Up at V1D0", "Down at V1D0")),
  row_title_rot   = 0,
  cluster_columns = FALSE,
  top_annotation  = top_anno_top50,
  column_names_side     = "top",
  column_names_rot      = 0,
  column_names_centered = TRUE,
  column_names_gp       = gpar(fontsize = 14, fontface = "bold"),
  row_names_gp          = gpar(fontsize = 12),
  row_names_max_width   = unit(70, "mm"),
  column_gap            = unit(4, "mm"),
  heatmap_legend_param = list(
    title = "ΔVST (SX − HC, row z)",
    legend_direction = "horizontal",
    at = c(-2, 0, 2),
    labels = c("-2", "0", "2"),
    title_gp  = gpar(fontface = "bold"),
    labels_gp = gpar(fontsize = 10)
  )
)

pdf("results_summary/summary_heatmap_topVar100_split_upDown_V1D0.pdf",
    width = 8, height = 15)
grid::grid.newpage()
draw(ht_top50, heatmap_legend_side = "bottom")
dev.off()

png("results_summary/summary_heatmap_topVar100_split_upDown_V1D0.png",
    width = 8, height = 15, units = "in", res = 700)
grid::grid.newpage()
draw(ht_top50, heatmap_legend_side = "bottom")
dev.off()


Warning message in gzfile(file, "rb"):
“cannot open compressed file 'results_paired_all_timepoints/se_paired_all_timepoints.rds', probable reason 'No such file or directory'”


ERROR: Error in gzfile(file, "rb"): cannot open the connection
